# ClusterAI 2020

# Ciencia de Datos - Ingeniería Industrial - UTN BA

# clase_10: Practica Redes Neuronales

### Elaborado por: Aguirre Nicolas

# IMPORTS

In [ ]:
import tensorflow as tf #Libreria de Redes Neronales
print(tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
pd.set_option('display.float_format', lambda x: '%.1d' % x) # Para acotar los decimales en pandas

# IRIS DATASET

En esta primera ejercitacion vamos a retomar el dataset Iris (visto en la Clase 03)

Recordemos que es un dataset de 3 clases de flores. De cada clase tenemos 50 muestras y 4 features.

El objetivo es clasificar las flores utilizando las 4 features utilizando redes neuronales. 



In [ ]:
# Primero cargamos los datos que ya vienen incluidos en la libreria sk-learn.
from sklearn.datasets import load_iris
iris = load_iris()

X = iris.data
Y = iris.target

In [ ]:
n_features = np.shape(X)[0]
n_samples = np.shape(X)[1]
n_classes = np.unique(Y)
print(f'Features: ',n_samples)
print(f'Samples: ',n_features)
print(f'Classes: ',n_classes)

**[0]**    SepalLengthCm            Length of the sepal (cm)

**[1]**    SepalWidthCm             Width of the sepal (cm)

**[2]**    PetalLengthCm            Length of the petal (cm)

**[3]**    PetalWidthCm             Width of the petal (cm)

In [ ]:
# Veamos la primera sample
print(f'X: {X[0]}')
print(f'Y: {Y[0]}')

## SPLIT

In [ ]:
# Separamos train y test set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## SCALING

In [ ]:
# Noralizamos
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train_norm = scaler.transform(x_train)
x_test_norm = scaler.transform(x_test)

## MODELO

Dentro de la libreria TensorFlow y su API Keras, existen dinstas manera de definir los modelos de Redes Neuronales (NN). El mas directo y sensillo es el modelo 'Sequential'.

En él, el foreward pass se hace, como su nombre lo indica, de manera secuencial con respecto a las capas (layers) que vamos agregando.

Para crear el modelo podemos hacerlo usando el metodo '.add()' o directamente pasar las layer dentro de una lista al momento de llamar a 'Sequential'

**.add()**
```
model = tf.keras.models.Sequential()
# Input Layer
model.add(tf.keras.layers.Dense(D_1,input_dim=4, activation='ZZZ'))

# Hidden Layers
model.add(tf.keras.layers.Dense(D_2, activation='ZZZ'))
...
model.add(tf.keras.layers.Dense(D_n, activation='ZZZ'))


# Output Layer
model.add(tf.keras.layers.Dense(D_o, activation='ZZZ'))

```
**Lista de layers**

```
model = tf.keras.models.Sequential([
  
  # Input Layer
  tf.keras.layers.Dense(D_1,input_dim=4, activation='ZZZ'),

  # Hidden Layers
  tf.keras.layers.Dense(D_2, activation='ZZZ'), 
  tf.keras.layers.Dense(D_3 activation='ZZZ'),
  ...

# Output Layer
  tf.keras.layers.Dense(D_o, activation='ZZZ')
])
```




In [ ]:
tf.keras.backend.clear_session() # Para eliminar los modelos que hayan quedado guardados en memoria

model = tf.keras.models.Sequential([
  # Primera capa de la red
  tf.keras.layers.Dense(4,input_dim=4, activation='sigmoid'),
  # Hidden Layers
  tf.keras.layers.Dense(4, activation='sigmoid'), 
  tf.keras.layers.Dense(4, activation='sigmoid'),
  # Salida de la red
  tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
#Veamos la arquitectura de nuestro modelo
model.summary()

**Consultas?**

### Ejemplo de salida del modelo

In [ ]:
# Imaginemos ahora 2 muestras
y_true_example = [0, 2]

# Para las cuales la red genera la siguiente salida del 'softmax'

y_pred_example = [[0.5, 0.25, 0.25], # Bien Clasificado
                  [0, 0.51, 0.49]] # Mal Clasificado

#Veamos como varia la loss en base a las predicciones de la red
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(reduction='none')
# El None en la reduccion, es para obtener la loss individualmente.
# Tiene otros usos, pero escapan al curso.

# Ahora veamos como daria la loss en cada caso:
loss_func(y_true_example, y_pred_example).numpy()

**Consultas?**

## OPTIMIZER & LOSS FUNCTION

In [ ]:
# Optimizador
lr = 0.001
opt = tf.keras.optimizers.SGD(learning_rate=lr)
# Funcion de penalizacion
# Usar esta cuando tenems muchas clases
loss_func = tf.keras.losses.SparseCategoricalCrossentropy() 

## COMPILE

In [ ]:
# Hasta este momento, hemos definido los elementos que conformaran el modelo.
# [arquitectura, optimizer, loss function].
# Internamente, estuvimos armando un 'grafo computacional', el cual debe
# compilarse.
model.compile(optimizer=opt,
              loss=loss_func,
              metrics=['accuracy'])

## TRAINING

In [ ]:
# Batch Size
bs = 8
# Epochs de entrenamiento
epochs_training = 300
# Entrenamos!
training = model.fit(x_train_norm,y_train,epochs=epochs_training,validation_split=0.2)

## HISTORY

El historial de entrenamiento queda guardado dentro de un diccionario en
training.history.

Deseamos ver como fue el progreso de neustro entrenamiento epochs por epoch en terminos de la loss_fuction y el accuracy.

Si al momento de compilar el modelo le pasamos mas metricas, quedan guardadas tambien.

In [ ]:
# El historial de entrenamiento quedó guardado en 
# 'history.history', ingresamos como 'key' cada una de las metricas que queremos plotear.

#Loss
loss_history = training.history['loss']
val_loss_hist = training.history['val_loss']
epochs = range(1, len(loss_history) + 1)
plt.plot(epochs, loss_history, 'r', label='Training loss')
plt.plot(epochs, val_loss_hist, 'b', label='Validation loss')
plt.title('Training and validation Loss')
plt.legend()
plt.show()

#Accuracy
acc_history = training.history['accuracy']
val_acc_hist = training.history['val_accuracy']
epochs = range(1, len(acc_history) + 1)
plt.plot(epochs, acc_history, 'r', label='Training Acc.')
plt.plot(epochs, val_acc_hist, 'b', label='Validation Acc.')
plt.title('Training and validation Acc.')
plt.legend()
plt.show()

## TEST

In [ ]:
model.evaluate(x_test_norm, y_test,verbose=2)

## CONFUSION MATRIX

In [ ]:
y_hat = model.predict(x_test_norm) # Salida de la red
y_pred = np.argmax(y_hat, axis=1) # Clase de la salida

In [ ]:
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index = np.arange(0,3),columns = np.arange(0,3))
plt.figure(figsize = (10,10))
sns.heatmap(df_cm, annot=True, cmap='jet',fmt='g')
plt.show()

## **PREGUNTAS**:
```
1) Es correcto que nuestra loss de train y de validacion disminuyan, y sin embargo el accuracy se mantenga igual/baje ? Por que ? 

2) Que cambios podriamos hacer para intentar solucionar el entrenamiento con la sigmoid function?
```



# MNIST DATASET

**Este dataset esta compuesto por 70.000 imagenes digitos (28 x 28 pixeles) del 0 al 9.**

**El objetivo es claisifcar correctamente los digitos de cada imagen.**

In [ ]:
# Cargamos el dataset desde la libreria de TF
mnist = tf.keras.datasets.mnist

In [ ]:
# Viene previamente dividido en train y test 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(f'Shape del x_train: {np.shape(x_train)}')
print(f'Shape del y_train: {np.shape(y_train)}')
print(f'Shape del x_test: {np.shape(x_test)}')

In [ ]:
# Veamos algunas muestras
sample = np.random.randint(0,q_train)
plt.figure(figsize=(10,10))
for i in range(sample,sample+25):
    plt.subplot(5,5,i-sample+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap='gray')
    plt.xlabel(y_train[i])
plt.show()

## SCALING

In [ ]:
# El valor esta en el rango [0-255]
# Vamos a pasarlos al rango [0-1]
x_train = x_train / 255.0
x_test = x_test / 255.0

## MODELO

In [ ]:
# Definicion del modelo:
model = tf.keras.models.Sequential([
  # Input layer
  tf.keras.layers.Flatten(input_shape=(28, 28)), # Pasamos de un input de forma 1x28x28 --> 1x784 
  
  # Hidden Layers
    #######################
    #------COMPLETAR------#
    #######################
  
  # Output Layers
    #######################
    #------COMPLETAR------#
    #######################
])

## OPTIMIZER, LOSS FUNCTION & COMPILE

In [ ]:
# Otpimizador
  #######################
  #------COMPLETAR------#
  #######################

# Loss Function
  #######################
  #------COMPLETAR------#
  #######################

# Compilar
model.compile(
    
  #######################
  #------COMPLETAR------#
  #######################
)  

## TRAINING

In [ ]:
# Entrenamiento
# Batch Size & Epochs
bs = 
epochs_train = 
  #######################
  #------COMPLETAR------#
  #######################
training = 

## HISTORY

In [ ]:
#######################
#------COMPLETAR------#
#######################

## TEST

In [ ]:
#######################
#------COMPLETAR------#
#######################

## CONFUSION MATRIX

In [ ]:
y_hat = model.predict(x_test) # Salida de la red
y_pred = np.argmax(y_hat, axis=1) # Clase de la salida

In [ ]:
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index = np.arange(0,10),columns = np.arange(0,10))
plt.figure(figsize = (15,15))
sns.heatmap(df_cm, annot=True, cmap='jet',fmt='g')
plt.show()